In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

In [3]:
# fonction pour récupérer le chemin absolu d'un fichier placé au même endroit que le notebook en cours
def get_output_path_file(output_filename):
    notebook_filename = os.path.abspath("__file__")
    notebook_directory = os.path.dirname(notebook_filename)
    return os.path.join(notebook_directory, output_filename)

# fonction pour récupérer le chemin absolu d'un dossier placé au même endroit que le notebook en cours
def get_output_path_folder(output_foldername):
    notebook_filename = os.path.abspath("__file__")
    notebook_directory = os.path.dirname(notebook_filename)
    return os.path.join(notebook_directory, output_foldername)

In [4]:
from joblib import load
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
import os
import pandas as pd
import time

def test_randomized_model(joblib_path_suivi_metrique_1, param_grid, model, randomized_search_CV):

    param_grid_df =[]
    for keys in param_grid:
        param_grid_df.append(keys)
     
    if os.path.exists(joblib_path_suivi_metrique_1):
        df_import_1 = load(joblib_path_suivi_metrique_1)
        print("Récupération du DataFrames existant")
    else:
        # Mettre à jour avec les paramètres pertinents pour XGBoost
        df_import_1 = pd.DataFrame(columns=param_grid_df + ['LogLoss_mean', 'LogLoss_std'])
        print("Création du nouveau DataFrame")
        
    print("Début du randomized:", randomized_search_CV)
    debut = time.time()
    randomized_search_CV.fit(X_train, y_train)
    print("Fin du randomized:", randomized_search_CV)
    fin = time.time()
    duree = fin - debut
    print(duree)
    results = randomized_search_CV.cv_results_

    dico_param_grid = {param_grid_df[i]:results.get("param_"+param_grid_df[i]) for i in range(len(param_grid_df))}
    dico_metric = {'LogLoss_mean': results.get('mean_test_score'),
                   'LogLoss_std': results.get('std_test_score')}
    
    combined_dict = {**dico_param_grid, **dico_metric}

    df_score_random = pd.DataFrame(combined_dict)
    
    df_score_random = pd.concat([df_import_1, df_score_random], ignore_index=True)
    
    return df_score_random
